In [215]:
#http://www.wildml.com/2015/09/implementing-a-neural-network-from-scratch/
import numpy as np
import sklearn
from sklearn import datasets, linear_model
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler,scale

import pandas as pd
%matplotlib inline  

In [216]:
TRAINING_RATE = 1
TESTING_RATE = 1 - TRAINING_RATE
MISSING_RATE = 0.5
QUERY_RATE = 0.2
# np.random.seed(10)

In [231]:
# Generate a dataset and plot it
np.random.seed(0)
#X : array of shape [n_samples, 2]
# e.g. X[0,:]  is [0.74, 0.46]
# X, y = sklearn.datasets.load_files("../dat/wine_quality") # X is 2 x 1 vector for 200 n_samples , y is 0 or 1
data = pd.read_csv("../dat/wine_quality/winequality-white.csv",sep=';')
# plt.scatter(X[:,0], X[:,1], s=40, c=y, cmap=plt.cm.Spectral)

In [257]:
def preprocessing(d):
    #Add normalization code here if necessary
    min_max_scaler = MinMaxScaler()
#     d.ix[:,0:-1] = scale(d.ix[:,0:-1])
    d.ix[:,0:-1] = min_max_scaler.fit_transform(d.ix[:,0:-1])
    #d['quality'] = d['quality'].apply(lambda x:1.0 if x==6 else 0.0)
    d['quality'] = d['quality'].apply(lambda x: int(x) -3)
    d = d.iloc[np.random.permutation(len(d))]
    t = int(len(d) * TRAINING_RATE)
    tn_data = d.iloc[0:t,:]
    tt_data = d.iloc[t:,:]
    
    tn_X = tn_data.ix[:,0:-1]
    tn_Y = tn_data.ix[:,-1]
    tt_X = tt_data.ix[:,0:-1]
    tt_Y = tt_data.ix[:,-1]
    
    
    return tn_X,tn_Y, tt_X, tt_Y

In [258]:
# tn_X,tn_Y,tt_X,tt_Y = preprocessing(data)
X, y,tt_X,tt_Y = preprocessing(data)
X = X.reset_index(drop=True)
X.head(15)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,0.403846,0.156863,0.216867,0.013804,0.112760,0.052265,0.278422,0.148255,0.436364,0.372093,0.322581
1,0.288462,0.215686,0.144578,0.092025,0.338279,0.114983,0.248260,0.157895,0.318182,0.302326,0.225806
2,0.259615,0.127451,0.210843,0.078221,0.100890,0.156794,0.436195,0.131290,0.472727,0.325581,0.338710
3,0.144231,0.127451,0.174699,0.001534,0.056380,0.031359,0.132251,0.097166,0.527273,0.209302,0.290323
4,0.230769,0.294118,0.253012,0.029141,0.086053,0.111498,0.250580,0.081164,0.581818,0.430233,0.580645
5,0.403846,0.137255,0.253012,0.214724,0.103858,0.149826,0.357309,0.254290,0.445455,0.546512,0.096774
6,0.288462,0.068627,0.198795,0.062883,0.148368,0.101045,0.252900,0.163678,0.645455,0.197674,0.161290
7,0.240385,0.107843,0.198795,0.145706,0.160237,0.212544,0.287703,0.163871,0.127273,0.220930,0.177419
8,0.278846,0.176471,0.156627,0.052147,0.207715,0.116725,0.480278,0.163678,0.536364,0.534884,0.241935
9,0.307692,0.264706,0.102410,0.007669,0.118694,0.017422,0.255220,0.112975,0.372727,0.162791,0.274194


In [259]:
# X.ix[0,0]
X = X.as_matrix() # change type "DataFrame" (of Pandas) to numpy array

In [260]:
X

array([[ 0.40384615,  0.15686275,  0.21686747, ...,  0.43636364,
         0.37209302,  0.32258065],
       [ 0.28846154,  0.21568627,  0.14457831, ...,  0.31818182,
         0.30232558,  0.22580645],
       [ 0.25961538,  0.12745098,  0.21084337, ...,  0.47272727,
         0.3255814 ,  0.33870968],
       ..., 
       [ 0.29807692,  0.20588235,  0.13855422, ...,  0.40909091,
         0.25581395,  0.12903226],
       [ 0.375     ,  0.19607843,  0.1746988 , ...,  0.4       ,
         0.19767442,  0.35483871],
       [ 0.28846154,  0.02941176,  0.25301205, ...,  0.41818182,
         0.60465116,  0.53225806]])

In [261]:
# Helper function to predict an output (0 or 1)
def predict(model, x):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    # Forward propagation
    z1 = x.dot(W1) + b1
    a1 = sigmoid(z1)
    z2 = a1.dot(W2) + b2 # 200 x 2 -> (200 x 3) * (3 x 2)
    a2 = sigmoid(z2) # 200 x 2
    return np.argmax(a2, axis=1)

In [262]:
def calculate_loss(model):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    # Forward propagation
    z1 = X.dot(W1) + b1
    a1 = sigmoid(z1)
    z2 = a1.dot(W2) + b2
    a2 = sigmoid(z2)
    corect_probs = np.square(a2 - X) * (1/2)
    data_loss = np.sum(corect_probs)
    return (1.0/num_examples) * data_loss

In [263]:
# Helper function to plot a decision boundary.
# If you don't fully understand this function don't worry, it just generates the contour plot below.
def plot_decision_boundary(pred_func):
    # Set min and max values and give it some padding
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    h = 0.01
    # Generate a grid of points with distance h between them
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    # Predict the function value for the whole gid
    Z = pred_func(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    # Plot the contour and training examples
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Spectral)

In [264]:
num_examples = len(X)
nn_input_dim = 11
nn_output_dim = 11
epsilon = 0.01 #GD update
print(X.shape)

(4898, 11)


In [265]:
def sigmoid(x):                                        
    return 1 / (1 + np.exp(-x))

In [266]:
def build_model(nn_hdim, num_passes=1000, print_loss=False):
    np.random.seed() # X is 200 x 2
    W1 = np.random.randn(nn_input_dim, nn_hdim) / np.sqrt(nn_input_dim) # 2 x 3
    b1 = np.zeros((1, nn_hdim)) # 1 x 3
    W2 = np.random.randn(nn_hdim, nn_output_dim) / np.sqrt(nn_hdim) # 3 x 2
    b2 = np.zeros((1, nn_output_dim)) # 1 x 2
    
    model = {}
    
    # Gradient Descent in batch
    for i in range(0, num_passes):
        
        # Forward propagation
        z1 = X.dot(W1) + b1 #200 x 3 -> (200 x 2) dot (2 x 3)
        a1 = sigmoid(z1) # 200 x 3
        z2 = a1.dot(W2) + b2 # 200 x 2 -> (200 x 3) * (3 x 2)
        a2 = sigmoid(z2) # 200 x 2
        # Backpropagation
        delta2 = (a2 * (1 - a2)) * (a2 - X) # 200 x 2 -> (200 x 2) * (200 x 2) 
        dW2 = (a1.T).dot(delta2) # 3 x 2 -> (3 x 200) dot (200 x 2)
        db2 = np.sum(delta2, axis=0, keepdims=True) # ? x ?
        delta1 = (a1 * (1 - a1)) * (delta2.dot(W2.T)) # 200 x 3 -> (200 x 3) * ((200 x 2) dot (2 x 3))
        dW1 = (X.T).dot(delta1) # 2 x 3 -> (2 x 200) dot (200 x 3)
        db1 = np.sum(delta1, axis=0) # ? x ?
        # Gradient descent parameter update
        W1 += -epsilon * dW1
        b1 += -epsilon * db1
        W2 += -epsilon * dW2
        b2 += -epsilon * db2
        # Assign new parameters to the model
        model = { 'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}
        if print_loss and i % 100 == 0:
          print( "Loss after iteration %i: %f" %(i, calculate_loss(model)) )
    return model, a2

In [267]:
# Build a model with a 3-dimensional hidden layer
model, a2 = build_model(3, print_loss=True)
 
# Plot the decision boundary -> no use for autoencoder
# plot_decision_boundary(lambda x: predict(model, x))
# plt.title("Decision Boundary for hidden layer size 3")

Loss after iteration 0: 0.280142
Loss after iteration 100: 0.113761
Loss after iteration 200: 0.110930
Loss after iteration 300: 0.131266
Loss after iteration 400: 0.086886
Loss after iteration 500: 0.086524
Loss after iteration 600: 0.075433
Loss after iteration 700: 0.088773
Loss after iteration 800: 0.080344
Loss after iteration 900: 0.061464


In [268]:
a2

array([[ 0.39290564,  0.15858842,  0.21009036, ...,  0.10355898,
         0.16047613,  0.08408966],
       [ 0.40922848,  0.14038352,  0.21156659, ...,  0.06619653,
         0.12866923,  0.01921571],
       [ 0.36422099,  0.20140749,  0.21548129, ...,  0.13728056,
         0.18539498,  0.07649471],
       ..., 
       [ 0.36927183,  0.19647907,  0.22021987, ...,  0.09081342,
         0.15075101,  0.01019198],
       [ 0.38158401,  0.13542802,  0.19136123, ...,  0.10613965,
         0.1143503 ,  0.0931769 ],
       [ 0.37822762,  0.18961921,  0.21409694, ...,  0.16791815,
         0.22501925,  0.35492935]])

In [269]:
X

array([[ 0.40384615,  0.15686275,  0.21686747, ...,  0.43636364,
         0.37209302,  0.32258065],
       [ 0.28846154,  0.21568627,  0.14457831, ...,  0.31818182,
         0.30232558,  0.22580645],
       [ 0.25961538,  0.12745098,  0.21084337, ...,  0.47272727,
         0.3255814 ,  0.33870968],
       ..., 
       [ 0.29807692,  0.20588235,  0.13855422, ...,  0.40909091,
         0.25581395,  0.12903226],
       [ 0.375     ,  0.19607843,  0.1746988 , ...,  0.4       ,
         0.19767442,  0.35483871],
       [ 0.28846154,  0.02941176,  0.25301205, ...,  0.41818182,
         0.60465116,  0.53225806]])

In [270]:
# np.square(a2 - X) * (1/2)